# PyTorch mutual information neural estimation image-based tests

Image-based high-dimensional tests

In [ ]:
import sys
sys.path.append("../python")

In [ ]:
import numpy as np

In [ ]:
import torch
import torchkld
import mutinfo

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print("Device: " + device)
print(f"Devices count: {torch.cuda.device_count()}")

In [ ]:
from tqdm import tqdm

In [ ]:
from mutinfo.distributions.base import *

In [ ]:
from misc.modules import *
from misc.plots import *
from misc.utils import *

In [ ]:
config = {}

## Dataset

Experimental setup

In [ ]:
config["mutual_information"] = 10.0

config["dataset_type"] = "gaussians"
assert config["dataset_type"] in ["rectangles", "gaussians"]

config["min_delta"] = 0.2 # Minimum (normalized) width/height for rectangles to avoid singularity.

In [ ]:
config["image_width"]  = 16
config["image_height"] = 16

In [ ]:
from mutinfo.distributions.tools import mapped_multi_rv_frozen
from mutinfo.distributions.images.geometric import uniform_to_rectangle, draw_rectangle
from mutinfo.distributions.images.field import draw_field, symmetric_gaussian_field

if config["dataset_type"] == "rectangles":
    config["X_latent_dimension"] = 4
    config["Y_latent_dimension"] = 4

    def generation_mapping(x, y):
        x = draw_rectangle(uniform_to_rectangle(ndtr(x), config["min_delta"], config["min_delta"]), config["image_width"], config["image_height"])
        y = draw_rectangle(uniform_to_rectangle(ndtr(y), config["min_delta"], config["min_delta"]), config["image_width"], config["image_height"])
        
        return x, y

    random_variable = mapped_multi_rv_frozen(
        CorrelatedNormal(config["mutual_information"], config["X_latent_dimension"], config["Y_latent_dimension"], randomize_interactions=False),
        generation_mapping
    )
    
elif config["dataset_type"] == "gaussians":
    config["X_latent_dimension"] = 2
    config["Y_latent_dimension"] = 2

    def generation_mapping(x, y):
        x = draw_field(ndtr(x), symmetric_gaussian_field, (config["image_width"], config["image_height"]))
        y = draw_field(ndtr(y), symmetric_gaussian_field, (config["image_width"], config["image_height"]))
        
        return x, y

    random_variable = mapped_multi_rv_frozen(
        CorrelatedNormal(config["mutual_information"], config["X_latent_dimension"], config["Y_latent_dimension"], randomize_interactions=False),
        generation_mapping
    )

In [ ]:
config["n_samples"] = 10*1024
x, y = random_variable.rvs(config["n_samples"])

In [ ]:
plt.imshow(random_variable.rvs(10)[0][0])

## Estimating MI

Dataset and dataloader

In [ ]:
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(x, dtype=torch.float32)[:,None,...],
    torch.tensor(y, dtype=torch.float32)[:,None,...],
)

x, y = random_variable.rvs(config["n_samples"])
test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(x, dtype=torch.float32)[:,None,...],
    torch.tensor(y, dtype=torch.float32)[:,None,...],
)

In [ ]:
config["batch_size"] = 128
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
test_dataloader  = torch.utils.data.DataLoader(test_dataset,  batch_size=config["batch_size"], shuffle=False)

Model

In [ ]:
n_filters = 8
hidden_dimension = 128

model = BasicConv2dT(config["image_width"], config["image_width"], n_filters=n_filters, hidden_dimension=hidden_dimension).to(device)
total_parameters = sum(parameter.numel() for parameter in model.parameters())
print(f"Total parameters: {total_parameters}")

In [ ]:
config["n_parameters"] = total_parameters

Loss

In [ ]:
# Loss.
config["biased"] = False
config["ema_multiplier"] = 1.0e-2
config["marginalize"] = "product" # "permute", "product"

losses = {
    "DonskerVaradhan": torchkld.loss.DonskerVaradhanLoss(biased=config["biased"], ema_multiplier=config["ema_multiplier"]),
    "NWJ": torchkld.loss.NWJLoss(),
    "Nishiyama": torchkld.loss.NishiyamaLoss(),
    "InfoNCE": torchkld.loss.InfoNCELoss(),
}

config["loss_name"] = "InfoNCE"
loss = losses[config["loss_name"]]

Optimizer

In [ ]:
config["learning_rate"] = 1.0e-3
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

Training

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
# Total number of epochs.
config["n_epochs"] = 100

# Number of epochs used to average the estimate.
config["average_epochs"] = 20

In [ ]:
from collections import defaultdict
from IPython.display import clear_output
from tqdm import trange

history = defaultdict(list)
for epoch in trange(1, config["n_epochs"] + 1, mininterval=1):    
    # Training.
    for index, batch in enumerate(train_dataloader):
        x, y = batch
        batch_size = x.shape[0]

        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        T_joined   = model(x.to(device), y.to(device))
        T_marginal = model(x.to(device), y.to(device), marginalize=config["marginalize"])
        _loss = loss(T_joined, T_marginal)
        _loss.backward()
        
        optimizer.step()

    history["train_mutual_information"].append(
        model.get_mutual_information(
            train_dataloader,
            loss,
            device,
            marginalize=config["marginalize"],
        )
    )
    history["test_mutual_information"].append(
        model.get_mutual_information(
            test_dataloader,
            loss,
            device,
            marginalize=config["marginalize"],
        )
    )

    if epoch % 5 == 0:        
        clear_output(wait=True)
        plot_estimated_MI_trainig(config["mutual_information"], np.arange(1, epoch+1), history["train_mutual_information"])
        plot_estimated_MI_trainig(config["mutual_information"], np.arange(1, epoch+1), history["test_mutual_information"])
        print(f"Current estimate: {history['test_mutual_information'][-1]:.3f}")
        print(f"Running median: {np.median(history['test_mutual_information'][-config['average_epochs']:]):.3f}")

### Saving the results

In [ ]:
config

In [ ]:
from datetime import datetime

experiment_name = f"{config['dataset_type']}_{config['image_width']}x{config['image_height']}_{config['mutual_information']:.1f}_{config['n_samples']}__{datetime.now().strftime('%d-%b-%Y_%H:%M:%S')}"
print(experiment_name)

In [ ]:
import os
from pathlib import Path

data_path = Path(os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), "../../data/synthetic")))
experiment_path = data_path / f"{config['dataset_type']}_{config['image_width']}x{config['image_height']}" / config['loss_name'] / experiment_name

In [ ]:
save_results(history, config, experiment_path, average_epochs=config['average_epochs'])